In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

In [2]:
# library import 
import os, sys
import jaydebeapi as jdb
import pandas as pd
import numpy as np
import seaborn as sns
import datetime, time
import time
import joblib

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

In [4]:
# get parameters
args_mm = '202011'

In [5]:
def oil_reg_type_anl(conn): 
    sql = """
    SELECT  *
      FROM  NEW_BIGDATA.B_DA_OIL_REG_TYPE_ANL_DS
    """
    
    df = pd.read_sql(sql, conn)
    return df

In [6]:
def debit_retl(conn): 
    sql = """
    SELECT  DBR_REG_NO 
            , H_CODE HCODE 
      FROM  NEW_BIGDATA.DEBIT_RETL
    """

    df = pd.read_sql(sql, conn)
    return df

In [57]:
# 1. Load Data  
# oil_cust_reg_tmp = oil_reg_type_anl(conn)
oil_cust_reg_tmp = pd.read_csv("./b_da_oil_reg_type_anl_ds.csv", dtype = {'reg_no':'str'})

In [147]:
# debit_retl = debit_retl(conn)
debit_retl = pd.read_csv(r".\debit_retl.csv", dtype = {'reg_no':'str', 'h_code':'str'})
debit_retl = debit_retl[['reg_no','h_code']]

In [60]:
# 2. Data Analysis
member_max_use_cnt = oil_cust_reg_tmp.groupby("member_no", as_index = False).app_cnt.sum()
member_max_use_cnt.columns = ["member_no", "total_app_cnt"]

In [62]:
# 단골형 / 일반형
dangol_cust = oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80").loc[:,["reg_no", "member_no", "use_reg_cnt", "app_cnt" ,"max_reg_use_cnt"]]
reg_regul_dat = dangol_cust.merge(member_max_use_cnt, how = 'left', on = "member_no")

reg_dangol_cust_md = reg_regul_dat.copy()
reg_dangol_cust_md['max_reg_use_rto'] = \
reg_dangol_cust_md['max_reg_use_cnt'] / reg_dangol_cust_md['total_app_cnt']

reg_dangol_cust_md = reg_dangol_cust_md.drop(["reg_no", "member_no","app_cnt", "total_app_cnt"], axis = 1)
reg_dangol_cust_md.columns

# --------------- 모델 적용 --------------- #
model = joblib.load('./model/regular_rf_202011.pkl')

y_prob = model.predict_proba(reg_dangol_cust_md.values)
y_pred = (y_prob[:,1] >= 0.9).astype("int")

dangol_cust['result'] = y_pred

In [63]:
# 출퇴근 / 종일
com_all_dat = oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80")
                              .loc[:,["commute", "all_day"]]
com_all_dat['commute_rto'] = com_all_dat['commute'] / com_all_dat['all_day']

# --------------- 모델 적용 --------------- #
model = joblib.load('./model/holy_rf_202011.pkl')

y_prob = model.predict_proba(com_all_dat.values)
y_pred = (y_prob[:,1] >= 0.9).astype("int")

com_all_dat['result'] = y_pred

In [64]:
# 고액형 / 알뜰형
full_min_dat = oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80").loc[:,["avg_app_amt", "avg_oil_amt", "oil_prc_diff", "avg_oil_prc"]]

# --------------- 모델 적용 --------------- #
model = joblib.load('./model/full_min_rf_202011.pkl')

y_prob = model.predict_proba(full_min_dat.values)
y_pred = (y_prob[:,1] >= 0.9).astype("int")

full_min_dat['result'] = y_pred

In [65]:
# 주중형 / 주말형
holy_dat = oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80").loc[:,["weekday", "weekend"]]
holy_dat['holy_rto'] = holy_dat['weekend'] / (holy_dat['weekend'] + holy_dat['weekday'])

# --------------- 모델 적용 --------------- #
model = joblib.load('./model/holy_rf_202011.pkl')

y_prob = model.predict_proba(holy_dat.values)
y_pred = (y_prob[:,1] >= 0.9).astype("int")

holy_dat['result'] = y_pred

In [66]:
# 고객 성향 => 개별 방문 주유소 매핑

In [97]:
# 단골/일반형 비율 산출
dancol_cust_r = \
dangol_cust.groupby(['reg_no', 'result']).member_no.nunique().reset_index()\
           .rename(columns = {"result": "regular_pred", "member_no": "regular_pred_cnt"})\
           .pivot(index= 'reg_no', columns= 'regular_pred', values= 'regular_pred_cnt')\
           .fillna(0).rename(columns = {0 : "regular_n", 1: "regular_y"})

dancol_cust_r['regular_rto'] = dancol_cust_r['regular_y']/(dancol_cust_r['regular_n'] + dancol_cust_r['regular_y'])
dancol_cust_r.replace([np.inf, -np.inf], 0, inplace = True)
dancol_cust_r.head()

regular_pred,regular_n,regular_y,regular_rto
reg_no,,,
1010366062,0.0,1.0,1.000000
1012554724,2011.0,202.0,0.091279
1013808744,605.0,113.0,0.157382
1020492065,86.0,60.0,0.410959
1021392617,48.0,92.0,0.657143


In [104]:
# 출퇴근형/일상형 비율 산출
com_all_dat_r = \
pd.concat([com_all_dat, 
            oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80")
            .loc[:,["reg_no"]]], axis = 1)\
            .groupby(['reg_no', 'result']).commute_rto.count().reset_index()\
            .rename(columns = {"result": "com_all_pred", "commute_rto": "com_all_pred_cnt"})\
            .pivot(index= 'reg_no', columns= 'com_all_pred', values= 'com_all_pred_cnt')\
            .fillna(0).rename(columns = {0 : "all_day", 1: "commute"})

com_all_dat_r['commute_rto'] = com_all_dat_r['commute']/(com_all_dat_r['commute'] + com_all_dat_r['all_day'])
com_all_dat_r.replace([np.inf, -np.inf], 0, inplace = True)
com_all_dat_r.head()

com_all_pred,all_day,commute,commute_rto
reg_no,,,
1010366062,0.0,1.0,1.000000
1012554724,809.0,1404.0,0.634433
1013808744,284.0,434.0,0.604457
1020492065,52.0,94.0,0.643836
1021392617,56.0,84.0,0.600000


In [105]:
# 고액/알뜰형 비율 산출
full_min_dat_r = \
pd.concat([full_min_dat, 
            oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80")
            .loc[:,["reg_no"]]], axis = 1)\
            .groupby(['reg_no', 'result']).avg_app_amt.count().reset_index()\
            .rename(columns = {"result": "full_chip_pred", "avg_app_amt": "full_chip_pred_cnt"})\
            .pivot(index= 'reg_no', columns= 'full_chip_pred', values= 'full_chip_pred_cnt')\
            .fillna(0).rename(columns = {0 : "chip", 1: "full"})

full_min_dat_r['full_rto'] = full_min_dat_r['full']/(full_min_dat_r['full'] + full_min_dat_r['chip'])
full_min_dat_r.replace([np.inf, -np.inf], 0, inplace = True)
full_min_dat_r.head()

full_chip_pred,chip,full,full_rto
reg_no,,,
1010366062,1.0,0.0,0.000000
1012554724,1062.0,1151.0,0.520108
1013808744,312.0,406.0,0.565460
1020492065,61.0,85.0,0.582192
1021392617,67.0,73.0,0.521429


In [110]:
# 주중/주말형 비율 산출
holy_dat_r = \
pd.concat([holy_dat, 
            oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80")
            .loc[:,["reg_no"]]], axis = 1)\
            .groupby(['reg_no', 'result']).holy_rto.count().reset_index()\
            .rename(columns = {"result": "weekday_end_pred", "holy_rto": "weekday_end_pred_cnt"})\
            .pivot(index= 'reg_no', columns= 'weekday_end_pred', values= 'weekday_end_pred_cnt')\
            .fillna(0).rename(columns = {0 : "weekday", 1: "weekend"})

holy_dat_r['week_end_rto'] = holy_dat_r['weekend']/(holy_dat_r['weekend'] + holy_dat_r['weekday'])
holy_dat_r.replace([np.inf, -np.inf], 0, inplace = True)
holy_dat_r.head()

weekday_end_pred,weekday,weekend,week_end_rto
reg_no,,,
1010366062,1.0,0.0,0.000000
1012554724,1514.0,699.0,0.315861
1013808744,556.0,162.0,0.225627
1020492065,103.0,43.0,0.294521
1021392617,79.0,61.0,0.435714


In [125]:
# 각 유형 결합
df = dancol_cust_r.join(com_all_dat_r, how = 'inner')\
                  .join(full_min_dat_r, how = 'inner')\
                  .join(holy_dat_r, how = 'inner')

In [128]:
df = df[['regular_rto','commute_rto','full_rto','week_end_rto']]

In [140]:
df['regular_std_rto'] = df.median()[0]
df['commute_std_rto'] = df.median()[1]
df['full_std_rto'] = df.median()[2]
df['weekend_std_rto'] = df.median()[3]

In [151]:
debit_retl['gungu_cd'] = debit_retl.h_code.str.slice(0,5)

In [160]:
result_df = df.merge(debit_retl[['reg_no', 'gungu_cd']], left_index = True, right_on = 'reg_no')

In [ ]:
oil_cust_reg_tmp.query("app_cnt >=6 & avg_oil_amt >= 8 & avg_oil_amt <= 80")
                              .loc[:,["commute", "all_day"]]

In [169]:
oil_juyuso_sales = \
oil_cust_reg_tmp.groupby("reg_no")\
                .agg({"app_cnt":["sum", "mean"] ,"app_amt":"sum",
                      "avg_app_amt":"mean", "avg_oil_amt":"mean",
                      "avg_oil_prc":"mean", "oil_prc_diff":"mean"})

In [170]:
oil_juyuso_sales.columns = ["app_cnt", "avg_app_cnt", "app_amt", "avg_app_amt", "avg_oil_amt", "avg_oil_prc", "oil_prc_diff"]

In [173]:
oil_juyuso_sales.head()

,app_cnt,avg_app_cnt,app_amt,avg_app_amt,avg_oil_amt,avg_oil_prc,oil_prc_diff
reg_no,,,,,,,
1010366062,195,2.119565,14492380,62892.728261,51.930109,1239.913043,51.391304
1012554724,31875,10.106214,1852203085,57130.194039,49.280704,1182.239696,-20.134115
1013808744,12423,7.014681,990944529,64220.057030,55.452400,1196.744212,-9.385658
1020492065,3521,3.163522,281438817,63252.656783,54.479066,1187.737646,-0.508535
1021392617,7001,2.434284,399015209,52385.185327,44.772573,1186.281989,-65.666551


In [176]:
smary_tt_sales = \
result_df.merge(oil_juyuso_sales, how = 'left', left_on = 'reg_no', right_index = True)

In [178]:
smary_tt_sales['md_yearmonth'] = '202011' # args_mm

In [184]:
smary_tt_sales = \
smary_tt_sales[["md_yearmonth","reg_no", "gungu_cd", 
                "app_amt", "app_cnt", "avg_app_amt", 
                "avg_app_cnt", "avg_oil_amt", "avg_oil_prc", 
                "oil_prc_diff", "commute_rto", "full_rto", 
                "week_end_rto", "regular_rto","commute_std_rto", 
                "full_std_rto", "weekend_std_rto", "regular_std_rto"]]

In [ ]:
smary_tt_sales.to_sql('B_DA_OIL_REG_TYPE_ANL', con = conn, if_exists= 'replace', chunksize = 1000, index=False)    